In [2]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import pymongo
import pandas as pd
import datetime

In [3]:
#Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Get LATEST driver version for 90.0.4430
Trying to download new driver from https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\10pai\.wdm\drivers\chromedriver\win32\90.0.4430.24]


In [25]:
#Create mongodb connection
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

In [27]:
#Create Beautiful Soup object
soup = bs(html_string, 'html.parser')

In [ ]:
url = 
browser.visit(url)

In [ ]:
browser.quit()